In [1]:
# Thanks for @cointegrated - author of https://habrahabr.ru/post/346198/ 
import numpy as np
import requests
import pickle
import time
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [8]:
def get_art_by_id(pid):
    """ Download and parse an article from postimees.ee """
    # downloading
    r = requests.get('https://postimees.ee/' +str(pid) + '/')
    # parsing
    soup = BeautifulSoup(r.text, 'html5lib')
    doc = {}
    doc['id'] = pid
    if (soup.title.text[:3]=='404') or \
        ((soup.find("html", attrs={'lang':'ru'})==None) and (soup.find("html", attrs={'lang':'et'})==None)) or \
        soup.find("div", attrs={'class':'saadetud_sobrale_onclick'})!=None or \
        soup.find("div", attrs={'id':'kaart_area'})!=None or \
        soup.find("span", attrs={'id':'scrollDown'})!=None or \
        soup.find("section", attrs={'id':'fullScreenImage'})!=None or \
        soup.find("span", attrs={'class':'header-logo__section slab'})!=None or \
        soup.find("div", attrs={'class':'article-lead__item'})!=None:
        # These are the exceptions that I've got from the experience - special editions and so on
        # Unfortunately I did not find any special features of that pages in html
        doc['status'] = 'title_not_found'
    else:
        doc['status'] = 'ok'
        
        if soup.find("html", attrs={'lang':'ru'})!=None:
            doc['lang']='ru'
        else:
            doc['lang']='et'

        doc['title'] = soup.find("meta", property="og:title")["content"]   # Title
        text=''
        for i in soup.find_all('p',attrs={'class':''}):
            text = text + i.text + " "
        doc['text'] = text # Text from a few paragraphs
        doc['theme'] = soup.find("meta", property="article:section")["content"] # Theme
        doc['keywords'] = soup.find("meta", attrs={'name':'keywords'})['content'] # Keywords
        doc['time'] = soup.find("meta", property="article:modified_time")["content"].replace('+02:00','') # Date and time
        comcount = soup.find("span", attrs={'class':'article-comment-count'})
        doc['comcount'] = 0 if comcount==None else comcount.text  # Count of comments
        autor = soup.find("a", attrs={'class':'article-author__link'})
        doc['autor']=0 if autor==None else autor.text  # Author
        doc['full_url'] = '' if soup.find("link", attrs={'rel':'canonical'})==None \
                                else soup.find("link", attrs={'rel':'canonical'})['href']  
            # Full url, because I use a short url and we have redirecting after this. The first part of the full url 
            # may be used for categorization
    
    # Save file with result dictionary
    fname = r'files/' + str(pid) + '.pkl'
    with open(fname, 'wb') as f:
        pickle.dump(doc, f)

In [30]:
#if __name__ == '__main__':
#    p = Pool(1) # Did not work on my computer
#    docs = p.map(get_doc_by_id, np.arange(4367958,4367961))
counter = 0
for i in list(range(4250000,4372719)):  # From 20.09.2017 to 12.01.2018
    counter += 1
    if counter%200==0: # Sleep I use because there is blocking on postimees.ee.
        time.sleep(5)
    elif counter == 4999:
        time.sleep(180)
        counter = 0
    get_art_by_id(i)